Importo librerías.

In [3]:
import pandas as pd
import numpy as np 
import ast
import json
import warnings
import os
from langdetect import detect
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from pandas.io.parquet import to_parquet
from pandas import json_normalize
warnings.filterwarnings("ignore")

In [4]:
# Obtener la ruta actual
ruta_actual = os.getcwd()

El archivo no se puede abrir comúnmente, se adecúa para su correcta apertura.

In [5]:
ruta_review = os.path.join(ruta_actual, 'PI MLOps - STEAM', 'australian_user_reviews.json')
ruta_nueva_reviews = os.path.join(ruta_actual, 'PI MLOps - STEAM', 'reviews.json')

In [ ]:
with open(ruta_review, 'r', encoding='utf-8') as archivo:
    lineas = archivo.readlines()
texto = []
for linea in lineas:
    texto_de_linea = linea.replace("'", '"').replace('"', '').replace('user_id: ',  '"user_id": "').replace(', user_url: ',  '", "user_url": "').replace(', reviews:',  '", "reviews":').replace('funny:',  '"funny": "').replace(', posted:',  '", "posted": "').replace(', last_edited:',  '", "last_edited": "').replace(', item_id:',  '", "item_id": "').replace(', helpful:',  '", "helpful": "').replace(', recommend:',  '", "recommend": "').replace(', review:',  '", "review": "').replace('},', '"},').replace('}]', '"}]').replace("\\", "")
    texto.append(texto_de_linea)

archivo = open(ruta_nueva_reviews, "w", encoding='utf-8')
archivo.write('[' + '\n')
for i in range(len(texto)-1):
    archivo.write(texto[i] + ',')
archivo.write(texto[len(texto)-1])
archivo.write(']' + '\n')
archivo.close()

Creo un DataFrame con lo que creé.

In [6]:
df_reviews = pd.read_json(ruta_nueva_reviews, encoding='utf-8')

Recaudo información de él.

In [7]:
df_reviews.info

<bound method DataFrame.info of                  user_id                                           user_url  \
0      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
1                js41637               http://steamcommunity.com/id/js41637   
2              evcentric             http://steamcommunity.com/id/evcentric   
3                  doctr                 http://steamcommunity.com/id/doctr   
4              maplemage             http://steamcommunity.com/id/maplemage   
...                  ...                                                ...   
25794  76561198306599751  http://steamcommunity.com/profiles/76561198306...   
25795           Ghoustik              http://steamcommunity.com/id/Ghoustik   
25796  76561198310819422  http://steamcommunity.com/profiles/76561198310...   
25797  76561198312638244  http://steamcommunity.com/profiles/76561198312...   
25798        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   

                   

El DF tiene una columna "reviews" que está conformada por varias columnas dentro de él, entonces descomprimo dicha columna.

In [8]:
def extraer_reviews(df_reviews):
    # Aplana la columna 'reviews' utilizando json_normalize
    flattened_reviews = json_normalize(df_reviews['reviews'])

    # Agrega 'user_id' al principio de las columnas aplanadas
    user_reviews =  pd.concat([df_reviews['user_id'], flattened_reviews], axis=1)

    return user_reviews

# extrae las columnas 'reviews'
user_reviews = extraer_reviews(df_reviews)

# Muestra el DataFrame resultante
user_reviews

,user_id,0,1,2,3,4,5,6,7,8,9
0,76561197970982479,"{'funny': ' ', 'posted': ' Posted November 5, ...","{'funny': ' ', 'posted': ' Posted July 15, 201...","{'funny': ' ', 'posted': ' Posted April 21, 20...",None,None,None,None,None,None,None
1,js41637,"{'funny': ' ', 'posted': ' Posted June 24, 201...","{'funny': ' ', 'posted': ' Posted September 8,...","{'funny': ' ', 'posted': ' Posted November 29,...",None,None,None,None,None,None,None
2,evcentric,"{'funny': ' ', 'posted': ' Posted February 3.'...","{'funny': ' ', 'posted': ' Posted December 4, ...","{'funny': ' ', 'posted': ' Posted November 3, ...","{'funny': ' ', 'posted': ' Posted October 15, ...","{'funny': ' ', 'posted': ' Posted October 15, ...","{'funny': ' ', 'posted': ' Posted October 15, ...",None,None,None,None
3,doctr,"{'funny': ' ', 'posted': ' Posted October 14, ...","{'funny': ' ', 'posted': ' Posted July 28, 201...","{'funny': ' ', 'posted': ' Posted June 2, 2012...","{'funny': ' ', 'posted': ' Posted June 29, 201...","{'funny': ' ', 'posted': ' Posted November 22,...","{'funny': ' ', 'posted': ' Posted February 23,...",None,None,None,None
4,maplemage,"{'funny': ' 3 people found this review funny',...","{'funny': ' 1 person found this review funny',...","{'funny': ' 2 people found this review funny',...","{'funny': ' ', 'posted': ' Posted July 11, 201...",None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
25794,76561198306599751,"{'funny': ' ', 'posted': ' Posted May 31.', 'l...",None,None,None,None,None,None,None,None,None
25795,Ghoustik,"{'funny': ' ', 'posted': ' Posted June 17.', '...",None,None,None,None,None,None,None,None,None
25796,76561198310819422,"{'funny': ' 1 person found this review funny',...",None,None,None,None,None,None,None,None,None
25797,76561198312638244,"{'funny': ' ', 'posted': ' Posted July 21.', '...","{'funny': ' ', 'posted': ' Posted July 10.', '...","{'funny': ' ', 'posted': ' Posted July 10.', '...","{'funny': ' ', 'posted': ' Posted July 8.', 'l...",None,None,None,None,None,None


In [9]:
#funcion para extrar columnas de review
def seleccionar_datos(df, fila, columna):
    # Seleccionar el dato en la fila y columna especificadas
    data_review = df.loc[fila, columna]
    # Crear un nuevo DataFrame con el dato seleccionado
    data = json_normalize(data_review)
    return data

In [10]:
ejemplo = seleccionar_datos(user_reviews, 0, 0)

ejemplo

,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my op...


Recorro el DF, fusiono "user_id" con su review en un nuevo DF separando las reviews en filas.

In [11]:
melted_df_reviews = pd.melt(user_reviews, id_vars=['user_id'], value_vars=[0,2,1,4,3,5,6,7,8,9,], var_name='Review', value_name='Review_Data')

melted_df_reviews

,user_id,Review,Review_Data
0,76561197970982479,0,"{'funny': ' ', 'posted': ' Posted November 5, ..."
1,js41637,0,"{'funny': ' ', 'posted': ' Posted June 24, 201..."
2,evcentric,0,"{'funny': ' ', 'posted': ' Posted February 3.'..."
3,doctr,0,"{'funny': ' ', 'posted': ' Posted October 14, ..."
4,maplemage,0,"{'funny': ' 3 people found this review funny',..."
...,...,...,...
257985,76561198306599751,9,None
257986,Ghoustik,9,None
257987,76561198310819422,9,None
257988,76561198312638244,9,None


Elimino reviews nulas (no aportan nada).

In [12]:
melted_df_reviews.dropna(subset=['Review_Data'], inplace=True)

melted_df_reviews

,user_id,Review,Review_Data
0,76561197970982479,0,"{'funny': ' ', 'posted': ' Posted November 5, ..."
1,js41637,0,"{'funny': ' ', 'posted': ' Posted June 24, 201..."
2,evcentric,0,"{'funny': ' ', 'posted': ' Posted February 3.'..."
3,doctr,0,"{'funny': ' ', 'posted': ' Posted October 14, ..."
4,maplemage,0,"{'funny': ' 3 people found this review funny',..."
...,...,...,...
256785,BonnieMTD,9,"{'funny': ' ', 'posted': ' Posted June 7, 2015..."
256904,amillionlemons,9,"{'funny': ' ', 'posted': ' Posted March 16, 20..."
257402,keepit1hunid,9,"{'funny': ' ', 'posted': ' Posted March 29, 20..."
257718,SKELETRONPRIMEISOP,9,"{'funny': ' ', 'posted': ' Posted August 9, 20..."


Ordeno el nuevo DF por "user_id".

In [13]:
melted_df_reviews.sort_values(by='user_id', inplace=True)

melted_df_reviews

,user_id,Review,Review_Data
8342,--000--,0,"{'funny': ' ', 'posted': ' Posted March 20, 20..."
25239,--ace--,0,"{'funny': ' ', 'posted': ' Posted January 24, ..."
76837,--ace--,1,"{'funny': ' ', 'posted': ' Posted May 30, 2014..."
64552,--ionex--,1,"{'funny': ' ', 'posted': ' Posted August 15, 2..."
12954,--ionex--,0,"{'funny': ' ', 'posted': ' Posted March 2.', '..."
...,...,...,...
75656,zyr0n1c,1,"{'funny': ' ', 'posted': ' Posted March 26, 20..."
178852,zyr0n1c,6,"{'funny': ' ', 'posted': ' Posted July 15, 201..."
127254,zyr0n1c,3,"{'funny': ' ', 'posted': ' Posted February 1, ..."
230450,zyr0n1c,8,"{'funny': ' ', 'posted': ' Posted December 23,..."


Normalizo "Review_Data".

In [14]:
df_reviews = melted_df_reviews['Review_Data'].apply(pd.Series, dtype='object')
df_reviews.head()

,funny,posted,last_edited,item_id,helpful,recommend,review
8342,,"Posted March 20, 2014.",,1250,1 of 1 people (100%) found this review helpful,True,หนุกคับ แนะนำ 10/10
25239,,"Posted January 24, 2014.",,440,2 of 3 people (67%) found this review helpful,True,the best game i ever plllayed
76837,,"Posted May 30, 2014.",,113200,0 of 1 people (0%) found this review helpful,True,One Of The Funnyest Games That Is Animated :)...
64552,,"Posted August 15, 2015.",,105600,No ratings yet,True,"Its an amazing game, and im glad that mac use..."
12954,,Posted March 2.,,730,No ratings yet,True,"it done brokeded on me, the game no longer wo..."


Como se ve, no tiene la columna "user_id" entonces la agrego.

In [15]:
df_reviews = pd.concat([melted_df_reviews[['user_id']], df_reviews], axis=1)
df_reviews

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
8342,--000--,,"Posted March 20, 2014.",,1250,1 of 1 people (100%) found this review helpful,True,หนุกคับ แนะนำ 10/10
25239,--ace--,,"Posted January 24, 2014.",,440,2 of 3 people (67%) found this review helpful,True,the best game i ever plllayed
76837,--ace--,,"Posted May 30, 2014.",,113200,0 of 1 people (0%) found this review helpful,True,One Of The Funnyest Games That Is Animated :)...
64552,--ionex--,,"Posted August 15, 2015.",,105600,No ratings yet,True,"Its an amazing game, and im glad that mac use..."
12954,--ionex--,,Posted March 2.,,730,No ratings yet,True,"it done brokeded on me, the game no longer wo..."
...,...,...,...,...,...,...,...,...
75656,zyr0n1c,,"Posted March 26, 2015.",,230410,1 of 1 people (100%) found this review helpful,True,Fantastic game! Lots of gamemodes and large v...
178852,zyr0n1c,,"Posted July 15, 2013.",,4000,0 of 1 people (0%) found this review helpful,True,Great Game! With lots and lots of props to pl...
127254,zyr0n1c,,"Posted February 1, 2015.",,730,2 of 5 people (40%) found this review helpful,True,After playing 500 hours on this fantastic FPS...
230450,zyr0n1c,,"Posted December 23, 2013.","Last edited March 10, 2015.",8980,0 of 4 people (0%) found this review helpful,True,A Gem.Its level of stupidity is just overwhem...


Las columnas "funny" y "last_edited" no me interesan entonces las elimino.

In [16]:
df_reviews = df_reviews.drop(columns=['funny', 'last_edited'])
df_reviews.columns

Index(['user_id', 'posted', 'item_id', 'helpful', 'recommend', 'review'], dtype='object')

In [17]:
#'reviews_posted' sea de tipo datetime
df_reviews['reviews_posted'] = pd.to_datetime(df_reviews['posted'].astype(str).str.replace(r'Posted |,|\.', '', regex=True), errors='coerce')

#Creo la columna 'year' a partir de 'reviews_posted'
df_reviews['year'] = df_reviews['reviews_posted'].dt.year.astype('Int64')

#Ordeno el DataFrame por 'item_id' y 'year' para asegurar que la interpolación se haga correctamente
df_reviews = df_reviews.sort_values(['item_id', 'year'])

In [18]:
df_reviews

,user_id,posted,item_id,helpful,recommend,review,reviews_posted,year
27915,76561198040188061,"Posted May 18, 2011.",10,0 of 2 people (0%) found this review helpful,True,this game is the 1# online action game is awe...,2011-05-18,2011
137939,maddoxx789,"Posted July 24, 2012.",10,No ratings yet,True,GYERTEK GAMELNI MINDENKI ITT VAN AKI SZÁMIT !...,2012-07-24,2012
143399,mixadance,"Posted September 27, 2012.",10,2 of 3 people (67%) found this review helpful,True,:D,2012-09-27,2012
17642,waspish,"Posted April 7, 2012.",10,No ratings yet,True,Good Game :D,2012-04-07,2012
69971,666_pl_Satan_pl_666,"Posted November 24, 2013.",10,2 of 2 people (100%) found this review helpful,True,"Ogólnie fajna gra, trudno nie polecić. Jedyny...",2013-11-24,2013
...,...,...,...,...,...,...,...,...
7202,shapes2004,Posted May 3.,99900,1 of 7 people (14%) found this review helpful,True,i only played it for the tf2 hat,NaT,<NA>
56454,Gatsukama,"Posted December 28, 2011.",99910,No ratings yet,True,"Looks like a cute childrens game, but has mor...",2011-12-28,2011
3512,WhenWeWereYoungggg,"Posted June 4, 2012.",99910,1 of 2 people (50%) found this review helpful,True,Super fun.,2012-06-04,2012
22577,Themfgamer,"Posted July 25, 2014.",99910,0 of 9 people (0%) found this review helpful,False,"1/10, just a bad game overall. Dead, Boring, ...",2014-07-25,2014


In [19]:
#Relleno valores nulos en 'year' mediante interpolación lineal por grupo (item_id	)
df_reviews['year'] = df_reviews.groupby('item_id', group_keys=False)['year'].apply(lambda group: group.interpolate(method='pad') if group.notna().any() else group)

#Si todavía hay valores nulos después de la interpolación, se llenan con la mediana.
df_reviews['year'] = df_reviews['year'].fillna(df_reviews['year'].median())

Comienzo con el análisis de sentimiento para futuramente crear el sistema de recomendación.

In [20]:
# Análisis de texto para determinar el idioma de las reseñas

def detectar_idioma(texto):
    try:
        return detect(texto)
    except:
        return None

# Aplicar la función para detectar idioma y crear una nueva columna 'language'
df_reviews['language'] = df_reviews['review'].apply(detectar_idioma)

# Calcular el conteo y porcentaje de cada idioma
conteo_por_idioma = df_reviews['language'].value_counts()
porcentaje_por_idioma = df_reviews['language'].value_counts(normalize=True) * 100

# Crear un nuevo DataFrame con el conteo y porcentaje
resumen_idiomas = pd.DataFrame({
    'Conteo': conteo_por_idioma,
    'Porcentaje': porcentaje_por_idioma.round(2).astype(str) + '%'
})

# Ordenar el DataFrame por el conteo de mayor a menor
resumen_idiomas = resumen_idiomas.sort_values(by='Conteo', ascending=False)
resumen_idiomas = resumen_idiomas.sort_values(by='Conteo', ascending=False)
resumen_idiomas.head()

,Conteo,Porcentaje
language,,
en,45715,77.88%
pt,2192,3.73%
es,1290,2.2%
de,1146,1.95%
so,1005,1.71%


A partir de ahora, preparo todo lo que servirá para el EDA.

In [21]:
# Mapeo de códigos de idioma a nombres completos (Top 5)
mapeo_idiomas = {
    'en': 'English',
    'pt': 'Portugués',
    'es': 'Español',
    'de': 'German',
    'so': 'Somali',
    # Agrega más mapeos según sea necesario
}

# Aplicar el mapeo al DataFrame
df_reviews['idioma_completo'] = df_reviews['language'].map(mapeo_idiomas)

In [22]:
# Filtrar los registros donde language no es igual a 'en'
ddf_reviews = df_reviews[df_reviews['idioma_completo'] == 'English']

# Borrar la columna 'idioma_completo'
ddf_reviews = ddf_reviews.drop('idioma_completo', axis=1)

In [23]:
#Se convierten todas las letras a minúsculas para asegurar que todas las palabras sean tratadas de la misma manera.
ddf_reviews.loc[:, 'review'] = ddf_reviews['review'].str.lower()

In [24]:
#Eliminación de caracteres especiales
ddf_reviews['review'] = ddf_reviews['review'].replace('[^A-Za-z0-9\s]+', '', regex=True)

In [25]:
#Elimina caracteres de puntuación que no aportan al análisis de sentimiento.
ddf_reviews.loc[:, 'review'] = ddf_reviews['review'].str.replace('[^\w\s]', '', regex=True)

In [26]:
# Análisis de sentimiento para categorizar las reseñas columna 'review'

def analyze_sentiments(df):
    # Instanciar el analizador de sentimientos
    sia = SentimentIntensityAnalyzer()

    # Aplicar el análisis de sentimientos y asignar valores numéricos
    df['compound_score'] = df['review'].apply(lambda review: sia.polarity_scores(review)['compound'])
    df['sentiment_analysis'] = df['compound_score'].apply(lambda score: 0 if score < 0 else (1 if score == 0 else 2))

    # Conteo de reviews por score
    score_counts = df['sentiment_analysis'].value_counts()

    # Conteo de reviews en blanco
    blank_reviews_count = df['review'].isnull().sum()

    # Total de reviews
    total_reviews = len(df)

    # Calcular porcentajes
    score_percentages = (score_counts / total_reviews * 100).round(2)
    blank_reviews_percentage = (blank_reviews_count / total_reviews * 100).round(2)

    return df, score_counts, blank_reviews_count, score_percentages, blank_reviews_percentage

In [27]:
# Llamar a la función analyze_sentiments
ddf_reviews, score_counts, blank_reviews_count, score_percentages, blank_reviews_percentage = analyze_sentiments(ddf_reviews)

In [28]:
# Se eliminan las columnas 'helpful', 'review' y 'compound_score', datos que no se necesitarán.
ddf_reviews.drop(['helpful', 'review','compound_score'], axis=1, inplace=True)

In [29]:
# Crear un nuevo DataFrame con el conteo y porcentaje
resumen_sentimientos = pd.DataFrame({
    'Conteo': score_counts,
    'Porcentaje': score_percentages.round(2).astype(str) + '%'
})

In [30]:
# Ordenar el DataFrame por el conteo de mayor a menor
resumen_sentimientos = resumen_sentimientos.sort_values(by='Conteo', ascending=False)

In [31]:
# Imprimir los resultados
print("\nResumen de análisis de sentimientos:")
print(resumen_sentimientos)
print("\nConteo de reviews en blanco: ", blank_reviews_count, " Porcentaje: ", blank_reviews_percentage.round(2).astype(str) + '%')


Resumen de análisis de sentimientos:
                    Conteo Porcentaje
sentiment_analysis                   
2                    32380     70.83%
0                     8538     18.68%
1                     4797     10.49%

Conteo de reviews en blanco:  0  Porcentaje:  0.0%


In [32]:
ddf_reviews

,user_id,posted,item_id,recommend,reviews_posted,year,language,sentiment_analysis
27915,76561198040188061,"Posted May 18, 2011.",10,True,2011-05-18,2011,en,2
121951,epic_doom,"Posted July 5, 2013.",10,True,2013-07-05,2013,en,1
3332,76561198072207162,"Posted January 23, 2014.",10,True,2014-01-23,2014,en,2
80514,BestinTheWorldThund3r,"Posted March 19, 2014.",10,True,2014-03-19,2014,en,2
56962,Bigzy,"Posted September 1, 2014.",10,True,2014-09-01,2014,en,2
...,...,...,...,...,...,...,...,...
79817,ironhorse612,Posted February 24.,99900,True,NaT,2015,en,2
7202,shapes2004,Posted May 3.,99900,True,NaT,2015,en,2
56454,Gatsukama,"Posted December 28, 2011.",99910,True,2011-12-28,2011,en,2
22577,Themfgamer,"Posted July 25, 2014.",99910,False,2014-07-25,2014,en,0


Elimino toda columna que no me sirva.

In [33]:
ddf_reviews = ddf_reviews.drop(['posted', 'language', 'reviews_posted'], axis=1)

ddf_reviews

,user_id,item_id,recommend,year,sentiment_analysis
27915,76561198040188061,10,True,2011,2
121951,epic_doom,10,True,2013,1
3332,76561198072207162,10,True,2014,2
80514,BestinTheWorldThund3r,10,True,2014,2
56962,Bigzy,10,True,2014,2
...,...,...,...,...,...
79817,ironhorse612,99900,True,2015,2
7202,shapes2004,99900,True,2015,2
56454,Gatsukama,99910,True,2011,2
22577,Themfgamer,99910,False,2014,0


In [34]:
# Cuenta los valores nulos por columna
valores_nulos = ddf_reviews.isnull().sum()

# Imprime el resultado
print("Valores nulos por columna:")
print(valores_nulos)

Valores nulos por columna:
user_id               0
item_id               0
recommend             0
year                  0
sentiment_analysis    0
dtype: int64


In [36]:
ddf_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45715 entries, 27915 to 90559
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             45715 non-null  object
 1   item_id             45715 non-null  object
 2   recommend           45715 non-null  object
 3   year                45715 non-null  Int64 
 4   sentiment_analysis  45715 non-null  int64 
dtypes: Int64(1), int64(1), object(3)
memory usage: 2.1+ MB


In [44]:
#Reemplazo true y false por 1s y 0s
ddf_reviews['recommend'] = ddf_reviews['recommend'].replace({'True': 1, ' True': 1, 'False': 0, ' False': 0})

In [45]:
#Cambio el tipo para que las columnas se adecúen mejor a lo que contienen.
ddf_reviews['item_id'] = ddf_reviews['item_id'].astype(int)
ddf_reviews['recommend'] = ddf_reviews['recommend'].astype(int)

In [46]:
ddf_reviews

,user_id,item_id,recommend,year,sentiment_analysis
27915,76561198040188061,10,1,2011,2
121951,epic_doom,10,1,2013,1
3332,76561198072207162,10,1,2014,2
80514,BestinTheWorldThund3r,10,1,2014,2
56962,Bigzy,10,1,2014,2
...,...,...,...,...,...
79817,ironhorse612,99900,1,2015,2
7202,shapes2004,99900,1,2015,2
56454,Gatsukama,99910,1,2011,2
22577,Themfgamer,99910,0,2014,0


In [47]:
ddf_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45715 entries, 27915 to 90559
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             45715 non-null  object
 1   item_id             45715 non-null  int32 
 2   recommend           45715 non-null  int32 
 3   year                45715 non-null  Int64 
 4   sentiment_analysis  45715 non-null  int64 
dtypes: Int64(1), int32(2), int64(1), object(1)
memory usage: 1.8+ MB


Exporto el DataFrame ya limpio, en tipo parquet para que sea menos pesado.

In [48]:
ruta_de_guardado = os.path.join(r'D:\1er Proyecto', 'datasets_finales', 'reviews.parquet')
ddf_reviews.to_parquet(ruta_de_guardado, index=False)